In [1]:
import numpy as np
import randrot as rd
import copy
import sys
sys.path.insert(0, '../')
from original import Kernels as K
import matplotlib.pyplot as plt



### Initial Definitions

In [ ]:
k3 = K.ThreeBodySingleSpecies(theta=[1., 10., 100000.])

Started compilation of theano three body single species kernels


In [ ]:
k3.theta = [.0001, .000001, 100000.]

In [ ]:

def k_tilde_basic(ri_n, rj_n, rij_n, rl_n, rm_n, rlm_n, sig):
    
    twosigsq = 2*sig**2
    k_cyc_1 = np.exp(-(ri_n-rl_n)**2/twosigsq) * np.exp(-(rj_n-rm_n)**2/twosigsq) * np.exp(-(rij_n-rlm_n)**2/twosigsq)
    
    k_cyc_2 = np.exp(-(ri_n-rm_n)**2/twosigsq) * np.exp(-(rj_n-rlm_n)**2/twosigsq) * np.exp(-(rij_n-rl_n)**2/twosigsq)
    
    k_cyc_3 = np.exp(-(ri_n-rlm_n)**2/twosigsq) * np.exp(-(rj_n-rl_n)**2/twosigsq) * np.exp(-(rij_n-rm_n)**2/twosigsq)
 
    return k_cyc_1 + k_cyc_2 + k_cyc_3

In [ ]:
# basic implementation 

def k3_basic(conf1, conf2, sig):

    k_ee = 0.
    
    for i in np.arange(conf1.shape[0]):
        for j in np.arange(i):
            for l in np.arange(conf2.shape[0]):
                for m in np.arange(conf2.shape[0]):
                    if l != m:
                        ri = conf1[i]
                        rj = conf1[j]
                        rij= rj - ri
                        rl = conf2[l]
                        rm = conf2[m]
                        rlm= rm - rl

                        rin = np.linalg.norm(ri)
                        rjn = np.linalg.norm(rj)
                        rijn= np.linalg.norm(rij)
                        rln = np.linalg.norm(rl)
                        rmn = np.linalg.norm(rm)
                        rlmn= np.linalg.norm(rlm)
                        
                        k_ee += k_tilde_basic(rin, rjn, rijn, rln, rmn, rlmn, sig)

    return k_ee

In [ ]:
# define a function taking the positions 
# of thre atoms and creating a local configuration around the first 

def build_conf(r1, r2, r3):
    r1, r2, r3 = np.array(r1, dtype = float), np.array(r2, dtype = float), np.array(r3, dtype = float)
    riv = r2[0:3]-r1[0:3]
    rjv = r3[0:3]-r1[0:3]

    ri = np.concatenate((riv, np.array([r1[3], r2[3]])))
    rj = np.concatenate((rjv, np.array([r1[3], r3[3]])))
    conf = np.reshape(np.array([[ri],[rj]]),(1, 2, 5))
    return conf
     

### A simple configuration

In [ ]:
# test that three body works correctly
s1 = 1.
s2 = 1.
s3 = 1.

In [ ]:
# Define thee atoms, symmetrically positioned

r1 = [0, 0, 0, s1]
r2 = [3, 0, 0, s2]
r3 = [0, 4, 0, s3]

c1 = build_conf(r1, r2, r3)
c1

In [ ]:
k3.calc_ee(c1, c1)

In [ ]:
# Define a random configuration

In [ ]:
confR = np.random.normal(0, 1, (3,3))
confR = np.hstack((confR, np.reshape([s1, s2, s3],(3,1))))
confR_ = build_conf(confR[0], confR[1], confR[2])


In [ ]:
print(confR)

In [ ]:
k3.calc_ee(confR_, confR_)

In [ ]:
k3_basic(confR_[0], confR_[0], .0000001)

In [ ]:
confRP = confR[:]

In [ ]:
confRP[0, :] = confR[2, :]
confRP[2, :] = confR[0, :]
confRP_ = build_conf(confRP[0], confRP[1], confRP[2])

In [ ]:
k3.calc_ee(confRP_, confRP_)

In [ ]:
k3_basic(confRP_[0], confRP_[0], .0000001)

### Randomly sampled triplets

In [ ]:
base_kernels = []

for i in np.arange(10):
    r1r = np.concatenate((np.random.normal(0, 1, (3,)),[s1]))
    r2r = np.concatenate((np.random.normal(0, 1, (3,)),[s2]))
    r3r = np.concatenate((np.random.normal(0, 1, (3,)), [s3]))
    c_ran = build_conf(r1r, r2r, r3r)
    base_kernels.append(k3.calc_ee(c_ran, c_ran)[0,0])

In [ ]:
base_kernels

In [ ]:
#plt.hist(base_kernels, 60)
#plt.show()

### Rndom Permutations

In [ ]:
np.random.permutation(confR)

In [ ]:
permuted_kernels = []

for i in np.arange(10):
    rs_per = np.random.permutation(confR)
    c_per = build_conf(rs_per[0], rs_per[1], rs_per[2])
    permuted_kernels.append(k3.calc_ee(c_per, c_per)[0,0])

In [ ]:
permuted_kernels

### Random Rotations

In [ ]:
rotated_kernels = []

for i in np.arange(10):
    RM = rd.generate(3)
    r1r, r2r, r3r = np.array(r1[:]), np.array(r2[:]), np.array(r3[:])
    r1r[0:3], r2r[0:3], r3r[0:3] = np.array(RM.dot(r1[0:3])), np.array(RM.dot(r2[0:3])), np.array(RM.dot(r3[0:3]))
    c_rot = build_conf(r1r, r2r, r3r)
    
    rotated_kernels.append(k3.calc_ee(c_rot, c_rot)[0,0])

In [ ]:
rotated_kernels